In [1]:
with open("part2_alt.csv") as infile:
    lines = infile.readlines()

header = lines.pop(0)
header

'Symbol,Side,Cxl,Qty,Price,Bid,Ask,T-DatS-DatTradeID,TradeTiSS,Exbkr,TradeCommType,SourceCommission,Account,OrderID,GTL,Trailer,Info,Clr\n'

Actually, due to the splitting of existing columns, the original col headers may not be as useful as I previously proposed, and thus will be discarded.

In [2]:
import csv # since csv was used to write data, it is useful for reading it
import re

In [3]:
output = []


line_reader2 = csv.reader(lines)

for line in line_reader2:
    
    line[3].replace(",", "")
    
    if len(line[4]) > 8: # indicates joined prices - normal ones are 2.4 or 2.365 or similar
        price1, price2 = line[4].split("0000")[:2]
        price2 = price2[price2.find(".")-1:] # trim off excess number - should only be one before decimal place
        line[4] = price1 + "0000"
        line.insert(5, price2 + "0000")
    
    enddigits = line[6][-5:]
    price3, dateblob = line[6].split("0000")[:2]
    date_extract = re.compile(r'(\d{2}/\d{2})(\d{2}/\d{2})(.+)')    
    date1, date2, idnum = date_extract.search(dateblob).groups()
    idnum += enddigits
    line[6] = price3
    line.insert(7, date1)
    line.insert(8, date2)
    line.insert(9, idnum)
    
    date_name = line[10].split(":")[-1]
    line.insert(11, date_name)
    line[10] = line[10][:line[10].find(date_name) -1]
    
    if line[12] != r"/17":
        to_move_right = line[12:18]
        to_move_left = line[18:23]
        
        line[12:17] = to_move_left
        line[16:-1] = to_move_right # yes, overwrite last element of the chunk moved left
    
    try: # sometimes there's an extra element at the end, sometimes just the part needing splitting
        _ = int(line[-1])
        last_item = line[-2].split("#")[-1]
        line.insert(-1, last_item)
        line[-3] = line[-3][:line[-3].find(last_item) -1]
    except:              
        last_item = line[-1].split("#")[-1]
        line.append(last_item)
        line[-2] = line[-2][:line[-2].find(last_item) -1]
    
    output.append(line)

#output

In [4]:
with open("final.csv", "w+") as of:
    writer = csv.writer(of)
    writer.writerows(output)